# Riak

In [1]:
import riak
from pprintpp import pprint as pp
import json
import pandas as pd

import uuid
import hashlib

## Lectura de información en Pandas

Partimos del dataset normalizado y lo desnormalizamos para guardarlo todo junto en RIAK

In [2]:
df_mov = pd.read_excel("../../data/black.xlsx", sheet_name= "Movimientos")
df_miembros = pd.read_excel("../../data/black.xlsx", sheet_name= "Miembros")
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 0 to 7623
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_miembro          7624 non-null   int64         
 1   fecha               7624 non-null   datetime64[ns]
 2   hora                7624 non-null   int64         
 3   minuto              7624 non-null   int64         
 4   importe             7624 non-null   float64       
 5   comercio            6823 non-null   object        
 6   actividad_completa  7623 non-null   object        
 7   actividad           7623 non-null   object        
 8   nombre              7624 non-null   object        
 9   funcion             7624 non-null   object        
 10  organizacion        6003 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 714.8+ KB


In [3]:
df.head(5)

,id_miembro,fecha,hora,minuto,importe,comercio,actividad_completa,actividad,nombre,funcion,organizacion
0,6,2007-03-21,15,26,3.20,PARKING EGUISA C/SEVILLA,GARAJES Y APARCAMIENTOS,COCHE,Antonio Romero Lázaro,concejal,PSOE
1,6,2006-11-23,20,44,75.86,POZUELO 4 S L,GASOLINERAS,COCHE,Antonio Romero Lázaro,concejal,PSOE
2,6,2005-09-03,16,40,91.37,IL PORTONE,RESTAURANTES RESTO,RESTAURANTE,Antonio Romero Lázaro,concejal,PSOE
3,6,2005-12-29,20,34,63.40,MUSGO,GRAN ALMACEN (A),COMPRA BIENES,Antonio Romero Lázaro,concejal,PSOE
4,6,2004-07-17,16,17,60.24,BERNABEU RIB S,RESTAURANTES RESTO,RESTAURANTE,Antonio Romero Lázaro,concejal,PSOE


## Riak ...

In [4]:
# connect to database
cliente = riak.RiakClient()
cliente.ping()

True

Utilizamos un bucket para guardar los movimientos en formato JSON y otro para guardar el importe acumulado por miembro.

En el bucket de movimientos utilizamos una clave autogenerada (no vamos a poder localizar los movimientos por clave) y en el bucket con el importe acumulado la clave es el nombre de la persona en formato HASH, para no tener problemas ni con los espacios del nombre ni con los acentos

Se van a crear los siguientes buckets:

|bucket|Clave|Contenido|
|-|-|-|
|movimientos|Marca temporal|Todos los datos del dataset en formnato JSON y desnormalizados|
|acum_importes|Código HASH de la persona que realiza el movimiento|Mapa con el nombre de la persona y el importe acumulado|

El bucket de tweets tendrá los siguientes índices:

|Nombre del índice|Contenido|¿Que busquedas permite hacer?|
|-|-|-|
|idx_miembro_bin|Nombre de la persona|Localizar movimientos de una persona concreta|

In [5]:
BUCKET_MOVIMIENTOS = 'movimientos'
BUCKET_ACUM_IMPORTES = 'acum_importes'

bucket_mov = cliente.bucket(BUCKET_MOVIMIENTOS)
bucket_acum_importes = cliente.bucket_type('maps').bucket(BUCKET_ACUM_IMPORTES)

Función para eliminar los datos de un BUCKET de Riak

In [6]:
def drop_keys(bucket):
    for keys in bucket.stream_keys():
        for key in keys:
            bucket.delete(key)

In [7]:
# Borramos los datos ..
drop_keys(bucket_mov)
drop_keys(bucket_acum_importes)

## Inserción de información en Riak

Para generar los datos en formato JSON partimos del DataFrame de Pandas y lo exportamos a JSON (en formato String) para luego cargarlo en un diccionario de Python

Observa que cuando se genera el dato en formato JSON, los campos fechas se almacenan como un [TIMESTAMP de unix](http://www.unixtimestamp.com), por lo que habría que volver a convertirlo a fecha según el caso

In [8]:
json_string = df.to_json(orient = 'records')
json_list = json.loads(json_string)

In [9]:
print(len(json_list))

7624


In [10]:
pp(json_list[0])

{
    'actividad': 'COCHE',
    'actividad_completa': 'GARAJES Y APARCAMIENTOS',
    'comercio': 'PARKING EGUISA C/SEVILLA',
    'fecha': 1174435200000,
    'funcion': 'concejal',
    'hora': 15,
    'id_miembro': 6,
    'importe': 3.2,
    'minuto': 26,
    'nombre': 'Antonio Romero Lázaro',
    'organizacion': 'PSOE',
}


Vamos a utilizar el nombre de la persona como clave, por lo que necesitamos convertirla previamente a un código que no lleve ni espacios ni acentos.

Este es precisamente el objetivo de la siguiente función ...

In [11]:
def hash_string(s):
    s_utf8 = s.encode('utf-8')
    return hashlib.md5(s_utf8).hexdigest()

Recorremos el dataset de movimientos, almacenando la información tanto en el bucket de movimientos como en el bucket que acumula los movimientos por cliente.

En este último utilizamos una funcionalidad que nos da Riak para guardar la información tipificada. Utilizamos un mapa que tiene dos elementos: Un registro donde guardamos el nombre del cliente y un contador donde guardamos el importe (sin decimales)

In [12]:
# Bucle de carga de datos ...
for movimiento_json in json_list:
    key = uuid.uuid4().hex
    hash_nombre =  hash_string(movimiento_json["nombre"])
    
    movimiento = bucket_mov.new(key, movimiento_json)
    movimiento.add_index('idx_miembro_bin', hash_nombre)
    movimiento.store()
    
    map_nombre = bucket_acum_importes.new(hash_nombre)
    map_nombre.registers['nombre'].assign(movimiento_json["nombre"])
    map_nombre.counters['importe'].increment(int(round(movimiento_json["importe"] * 100)))
    map_nombre.store()
    

In [36]:
for key in bucket_mov.get_keys()[0:3]:
    data = bucket_mov.get(key).data
    print('%s -> %s' % (key, data))

405c73ace96b476cabf12ef3908fa5a8 -> {'id_miembro': 61, 'fecha': 1171497600000, 'hora': 15, 'minuto': 49, 'importe': 21.52, 'comercio': 'CANAL DE EDITORIALES S A', 'actividad_completa': 'LIBRERIAS, PAPELERIAS Y DISCOS', 'actividad': 'LIBRERIA', 'nombre': 'Miguel Blesa de la Parra', 'funcion': 'directivo', 'organizacion': None}
3f7d2e9e3681457a80d66e36d0a389e7 -> {'id_miembro': 6, 'fecha': 1160179200000, 'hora': 16, 'minuto': 16, 'importe': 87.69, 'comercio': 'LA VINOTECA', 'actividad_completa': 'RESTAURANTES RESTO', 'actividad': 'RESTAURANTE', 'nombre': 'Antonio Romero Lázaro', 'funcion': 'concejal', 'organizacion': 'PSOE'}
3ddd1a13c7cb4e4a9e939950a982e771 -> {'id_miembro': 83, 'fecha': 1086480000000, 'hora': 17, 'minuto': 12, 'importe': 50.0, 'comercio': 'E S CONCORDY S A', 'actividad_completa': 'GASOLINERAS', 'actividad': 'COCHE', 'nombre': 'Ángel Eugenio Gómez del Pulgar Perales', 'funcion': 'concejal', 'organizacion': 'PSOE'}


In [37]:
for key in bucket_acum_importes.get_keys()[0:3]:
    data = bucket_acum_importes.get(key)
    print('%s -> %s' % (key, data))

ebb7326e90b94d482c381db357646c61 -> {('importe', 'counter'): 2126307, ('nombre', 'register'): 'José María de la Riva Amez'}
b1e48c4c41ce251603da76501a50a8ee -> {('importe', 'counter'): 565771, ('nombre', 'register'): 'María Elena Gil García'}
6b2926b312574fa88c283ecd25c877e3 -> {('importe', 'counter'): 1435402, ('nombre', 'register'): 'Juan Gómez Castañeda'}


# Lectura de información en Riak

En la siguiente función partimos del objeto JSON que nos devuelve RIAK y generamos un DataFrame Pandas.

En el caso de la fecha, lo convertimos desde un UNIX Timestamp al formato correcto para Pandas

In [38]:
def json_to_pandas(rows):
    # Convertimos el objeto JSON en un objeto pandas 
    df = pd.read_json(json.dumps(rows))
    
    if 'fecha' in df.columns:
        # Las fechas están en formato UNIX TIMESTAMP. Las volvemos a convertir a formato Date...
        df = df.assign(fecha = pd.to_datetime(df.fecha, unit = 'ms'))

    return df

### Lectura del bucket de movimientos

Cargamos todos los movimientos de la base de datos y lo guardamos en un objeto Pandas, ya que hay ciertas preguntas que no pueden resolverse directamente por esta base de datos

Recuerda que RIAK permite obtener información de un clave, pero no le es posible devolver la información ordenada ...

In [39]:
rows = []
for keys in bucket_mov.stream_keys():
    for key in keys:
        # print('Key %s' % key )
        rows.append(bucket_mov.get(key).data)
        
# Convertimos el objeto json en un objeto pandas 
df_movimientos = json_to_pandas(rows)

In [40]:
df_movimientos.head()

,id_miembro,fecha,hora,minuto,importe,comercio,actividad_completa,actividad,nombre,funcion,organizacion
0,20,2006-06-07,17,36,209.19,RESTAURANTE ERROTA ZAR,RESTAURANTES RESTO,RESTAURANTE,Francisco José Pérez Fernández,concejal,PSOE
1,54,2011-10-10,21,51,87.00,REST LA CARIHUELA CHICA,RESTAURANTES RESTO,RESTAURANTE,Maria Mercedes de la Merced Monge,concejal,Partido Popular
2,30,2010-03-31,23,47,14.00,MALLORCA,HIPERMERCADOS (TECNOLOGIA ELECTRONICA),SUPERMERCADO,Jesús Pedroche Nieto,concejal,Partido Popular
3,81,2010-04-09,23,51,175.30,EL MONTERO DE CAZORLA,RESTAURANTES RESTO,RESTAURANTE,Santiago Javier Sánchez Carlos,concejal,PSOE
4,5,2003-06-06,0,14,148.30,RESTAURANTE LA TRAINERA,RESTAURANTE 4/5 TENEDORES NIVEL 3,RESTAURANTE,Antonio Rey de Viñas Sánchez-Majestad,concejal,CC OO


### Los 10 movimientos mas caros

Esta consulta no la podemos contestar directamente con la Base de Datos por lo que nos apoyamos en un proceso en el cliente

In [42]:
df_movimientos \
    .sort_values('importe', ascending=False)\
    .filter(['nombre', 'fecha', 'actividad_completa', 'importe']) \
    .head(10)

,nombre,fecha,actividad_completa,importe
2812,Ildefonso José Sánchez Barcoj,2009-12-30,EL CORTE INGLES,16921.75
1935,Miguel Blesa de la Parra,2006-04-04,EL CORTE INGLES,12597.26
5004,Ramón Ferraz Ricarte,2003-04-03,AGENCIAS DE VIAJES,5998.35
1224,Ramón Ferraz Ricarte,2009-08-23,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",5909.29
2938,Estanislao Rodríguez-Ponga Salamanca,2010-12-20,HIPERCOR SUPERMERCADOS EL CORTE INGLES,4985.00
5354,Matías Amat Roca,2006-08-02,AGENCIAS DE VIAJES,4972.00
2018,María Carmen Cafranga Cavestany,2010-10-07,AGENCIAS DE VIAJES,4336.00
7528,Ildefonso José Sánchez Barcoj,2010-03-25,EL CORTE INGLES,4320.50
5497,Carmen Contreras Gómez,2008-12-19,JOYERIAS Y RELOJERIAS,4218.00
3840,Carmen Contreras Gómez,2007-02-11,AIR EUROPA,4175.87


### Lectura a través de un índice

In [48]:
rows = []
keys = bucket_mov.stream_index("idx_miembro_bin", hash_string(u"Ildefonso José Sánchez Barcoj"))
for keys in keys.results:
    for movimiento_key in keys:
        rows.append(bucket_mov.get(movimiento_key).data)
    
df = json_to_pandas(rows)

### Los movimientos de una persona concreta

Para obtener los movimientos de una persona en concreto si podemos utilizar el índice que habíamos creado Ad-hoc, aunque la ordenación se realiza en Pandas

In [49]:
df \
    .sort_values('importe', ascending=False) \
    .filter(['nombre', 'fecha', 'actividad_completa', 'importe']) \
    .head(10)

,nombre,fecha,actividad_completa,importe
309,Ildefonso José Sánchez Barcoj,2009-12-30,EL CORTE INGLES,16921.75
207,Ildefonso José Sánchez Barcoj,2010-03-25,EL CORTE INGLES,4320.50
74,Ildefonso José Sánchez Barcoj,2007-11-01,EL CORTE INGLES,3882.00
212,Ildefonso José Sánchez Barcoj,2009-07-28,EL CORTE INGLES,2750.00
216,Ildefonso José Sánchez Barcoj,2011-12-18,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),1400.00
137,Ildefonso José Sánchez Barcoj,2004-04-12,EL CORTE INGLES,1218.00
211,Ildefonso José Sánchez Barcoj,2008-12-07,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",1024.65
91,Ildefonso José Sánchez Barcoj,2005-11-23,EL CORTE INGLES,972.71
136,Ildefonso José Sánchez Barcoj,2012-03-15,IBERIA,940.00
194,Ildefonso José Sánchez Barcoj,2012-04-10,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),900.00


### Lectura de una información agregada

La información agregada está en un mapa, por lo que tenemos que procesarla y convertirla a formato JSON.

In [50]:
rows = []
for keys in bucket_acum_importes.stream_keys():
    #print(keys)
    for key in keys:
        map_nombre = bucket_acum_importes.get(key)
        rows.append({'nombre' : map_nombre.registers['nombre'].value,
                     "importe" : float(map_nombre.counters['importe'].value) / 100})
        
# Convertimos el objeto json en un objeto pandas 
df = json_to_pandas(rows)

### Las 10 personas que mas han gastado

Fácil si disponemos de un agregado. Utilizamos Pandas para ordenar la información ..,

In [51]:
df.sort_values('importe', ascending=False).head(10)

,nombre,importe
61,Ildefonso José Sánchez Barcoj,71331.33
51,Miguel Blesa de la Parra,60325.29
25,Ricardo Morado Iglesias,46435.74
46,José Antonio Moral Santín,45026.71
36,Ramón Ferraz Ricarte,44438.94
43,Matías Amat Roca,42663.76
52,Mariano Pérez Claver,32121.01
11,Francisco Baquero Noriega,31558.70
72,Carmen Contreras Gómez,30159.02
12,Antonio Romero Lázaro,27954.38
